In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.utils.data import TensorDataset
from torchvision import datasets, transforms
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
import numpy as np
import copy

In [2]:
# Train augmentations
transforms = transforms.Compose([
    # Add training augmentations here, remember: we do not want to transform the validation images.
    # For information about augmentation see: https://pytorch.org/vision/stable/transforms.html
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),

])

In [3]:
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)

cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)

100%|██████████| 170498071/170498071 [00:16<00:00, 10161899.92it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Spliting dataset
train_size= int(0.8 * len(cifar_trainset))
val_size = len(cifar_trainset) - train_size
train_dataset, val_dataset = random_split(cifar_trainset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(cifar_testset, batch_size=32, shuffle=False)


In [5]:
print(val_size)
print(len(cifar_testset))
images, labels = next(iter(train_loader))
print(images.shape)

10000
10000
torch.Size([32, 3, 32, 32])


In [6]:
input_channel = 3
output_channel = 10

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(input_channel,output_channel, kernel_size=3, stride=1, padding=1)
        self.relu = nn.LeakyReLU(negative_slope=0.01, inplace=False)
        self.maxpool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16*16*output_channel, 10)  # Adjust the input size here
        self.dropout = nn.Dropout(0.3, inplace=False)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)  # Flatten the output before passing to fully connected layer
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x



# Instantiate the CNN, loss function, and optimizer
net = CNN()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, weight_decay=1e-2)

In [7]:
epochs = 5


for epoch in range(epochs):

    print(epoch+1)
    # Training phase
    for batch_nr, (images, labels) in enumerate(train_loader):
        images = images.float()
        labels = labels.long()  # Ensure labels are long integers for CrossEntropyLoss

        # Forward pass
        outputs = net(images)  # Do not apply sigmoid here

        # Compute loss
        loss = loss_function(outputs, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()




1
2
3
4
5


In [9]:
def validation(model, device, val_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradients needed for testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 validation images: %d %%' % (100 * correct / total))

def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradients needed for testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))



In [10]:
validate_model = validation(net, 'cpu', val_loader)
test_model = test(net,'cpu', test_loader)

Accuracy of the network on the 10000 validation images: 29 %
Accuracy of the network on the 10000 test images: 29 %


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the pixel values
])

# Download and load the MNIST dataset
train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Define data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, loss function, and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 100 == 99:
            print(f'Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {running_loss / 100:.4f}')
            running_loss = 0.0


tested_model = test(model,'cpu',test_loader)

100%|██████████| 9912422/9912422 [00:00<00:00, 13941601.95it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 16825962.89it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw


100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



Epoch 1, Batch 100, Loss: 0.6880
Epoch 1, Batch 200, Loss: 0.1760
Epoch 1, Batch 300, Loss: 0.1279
Epoch 1, Batch 400, Loss: 0.0960
Epoch 1, Batch 500, Loss: 0.0896
Epoch 1, Batch 600, Loss: 0.0718
Epoch 1, Batch 700, Loss: 0.0758
Epoch 1, Batch 800, Loss: 0.0766
Epoch 1, Batch 900, Loss: 0.0628
Epoch 2, Batch 100, Loss: 0.0549
Epoch 2, Batch 200, Loss: 0.0580
Epoch 2, Batch 300, Loss: 0.0454
Epoch 2, Batch 400, Loss: 0.0507
Epoch 2, Batch 500, Loss: 0.0458
Epoch 2, Batch 600, Loss: 0.0416
Epoch 2, Batch 700, Loss: 0.0446
Epoch 2, Batch 800, Loss: 0.0444
Epoch 2, Batch 900, Loss: 0.0397
Epoch 3, Batch 100, Loss: 0.0225
Epoch 3, Batch 200, Loss: 0.0309
Epoch 3, Batch 300, Loss: 0.0306
Epoch 3, Batch 400, Loss: 0.0395
Epoch 3, Batch 500, Loss: 0.0343
Epoch 3, Batch 600, Loss: 0.0366
Epoch 3, Batch 700, Loss: 0.0311
Epoch 3, Batch 800, Loss: 0.0360
Epoch 3, Batch 900, Loss: 0.0367
Epoch 4, Batch 100, Loss: 0.0212
Epoch 4, Batch 200, Loss: 0.0226
Epoch 4, Batch 300, Loss: 0.0279
Epoch 4, B